# High-level MXNet Example

In [1]:
import os
import sys
import numpy as np
import mxnet as mx
from common.params import *
from common.utils import *

In [2]:
# Force one-gpu
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("MXNet: ", mx.__version__)
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Numpy:  1.14.1
MXNet:  0.12.0
GPU:  ['Tesla P100-PCIE-16GB', 'Tesla P100-PCIE-16GB']
CUDA Version 8.0.61
CuDNN Version  6.0.21


In [4]:
def create_symbol(n_classes=N_CLASSES):
    data = mx.symbol.Variable('data')
    # size = [(old-size - kernel + 2*padding)/stride]+1
    # if kernel = 3, pad with 1 either side
    conv1 = mx.symbol.Convolution(data=data, num_filter=50, pad=(1,1), kernel=(3,3))
    relu1 = mx.symbol.Activation(data=conv1, act_type="relu")
    conv2 = mx.symbol.Convolution(data=relu1, num_filter=50, pad=(1,1), kernel=(3,3))
    pool1 = mx.symbol.Pooling(data=conv2, pool_type="max", kernel=(2,2), stride=(2,2))
    relu2 = mx.symbol.Activation(data=pool1, act_type="relu")
    drop1 = mx.symbol.Dropout(data=relu2, p=0.25)
    
    conv3 = mx.symbol.Convolution(data=drop1, num_filter=100, pad=(1,1), kernel=(3,3))
    relu3 = mx.symbol.Activation(data=conv3, act_type="relu")
    conv4 = mx.symbol.Convolution(data=relu3, num_filter=100, pad=(1,1), kernel=(3,3))
    pool2 = mx.symbol.Pooling(data=conv4, pool_type="max", kernel=(2,2), stride=(2,2))
    relu4 = mx.symbol.Activation(data=pool2, act_type="relu")
    drop2 = mx.symbol.Dropout(data=relu4, p=0.25)
           
    flat1 = mx.symbol.Flatten(data=drop2)
    fc1 = mx.symbol.FullyConnected(data=flat1, num_hidden=512)
    relu7 = mx.symbol.Activation(data=fc1, act_type="relu")
    drop4 = mx.symbol.Dropout(data=relu7, p=0.5)
    fc2 = mx.symbol.FullyConnected(data=drop4, num_hidden=n_classes) 
    
    input_y = mx.symbol.Variable('softmax_label')  
    m = mx.symbol.SoftmaxOutput(data=fc2, label=input_y, name="softmax")
    return m

In [5]:
def init_model(m, batchsize=BATCHSIZE, lr=LR, momentum=MOMENTUM):
    ctx = [mx.gpu(0)]
    mod = mx.mod.Module(context=ctx, symbol=m)
    mod.bind(data_shapes=[('data', (batchsize, 3, 32, 32))],
             label_shapes=[('softmax_label', (batchsize,))])

    # Glorot-uniform initializer
    mod.init_params(initializer=mx.init.Xavier(rnd_type='uniform'))
    mod.init_optimizer(optimizer='sgd', 
                       optimizer_params=(('learning_rate', lr), ('momentum', momentum), ))
    return mod

In [6]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = cifar_for_library(channel_first=True)
# Load data-iterator
train_iter = mx.io.NDArrayIter(x_train, y_train, BATCHSIZE, shuffle=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...
Preparing test set...
(50000, 3, 32, 32) (10000, 3, 32, 32) (50000,) (10000,)
float32 float32 int32 int32
CPU times: user 1.05 s, sys: 1.09 s, total: 2.14 s
Wall time: 2.14 s


In [7]:
%%time
# Load symbol
sym = create_symbol()

CPU times: user 1.87 ms, sys: 382 µs, total: 2.25 ms
Wall time: 1.7 ms


In [8]:
%%time
# Initialise model
model = init_model(sym)

CPU times: user 926 ms, sys: 705 ms, total: 1.63 s
Wall time: 1.89 s


In [9]:
%%time
# Main training loop: 52s
metric = mx.metric.create('acc')
for j in range(EPOCHS):
    train_iter.reset()
    metric.reset()
    for batch in train_iter:
        model.forward(batch, is_train=True) 
        model.update_metric(metric, batch.label)
        model.backward()              
        model.update()
    print('Epoch %d, Training %s' % (j, metric.get()))

Epoch 0, Training ('accuracy', 0.3116807864450128)
Epoch 1, Training ('accuracy', 0.48479459718670076)
Epoch 2, Training ('accuracy', 0.5717710997442456)
Epoch 3, Training ('accuracy', 0.6322330562659847)
Epoch 4, Training ('accuracy', 0.6781289961636828)
Epoch 5, Training ('accuracy', 0.7113970588235294)
Epoch 6, Training ('accuracy', 0.7390704923273658)
Epoch 7, Training ('accuracy', 0.7605498721227621)
Epoch 8, Training ('accuracy', 0.7807904411764706)
Epoch 9, Training ('accuracy', 0.7980338874680307)
CPU times: user 47.1 s, sys: 14.3 s, total: 1min 1s
Wall time: 49.5 s


In [10]:
%%time
# Main evaluation loop: 311ms
y_guess = model.predict(mx.io.NDArrayIter(x_test, batch_size=BATCHSIZE, shuffle=False))
y_guess = np.argmax(y_guess.asnumpy(), axis=-1)

CPU times: user 359 ms, sys: 145 ms, total: 504 ms
Wall time: 310 ms


In [11]:
print("Accuracy: ", sum(y_guess == y_test)/len(y_guess))

Accuracy:  0.7674
